In [ ]:
"""!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install polars[numpy,pandas,pyarrow] --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg"""

In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import duckdb as dd
import polars as pl
import os
import glob
import shutil
import zipfile
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
#!pip install plotly
import plotly.express as px
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
#!pip install -U imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [17]:
def audio_waveframe(file_path):
    # Load the audio file
    audio_data, sampling_rate = librosa.load(file_path)
    # Calculate the duration of the audio file
    duration = len(audio_data) / sampling_rate
    # Create a time array for plotting
    time = np.arange(0, duration, 1/sampling_rate)
    # Plot the waveform
    plt.figure(figsize=(30, 4))
    plt.plot(time, audio_data, color='blue')
    plt.title('Audio Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plot = plt.show()
    return plot

def spectrogram(file_path):
    # Compute the short-time Fourier transform (STFT)
    n_fft = 500  # Number of FFT points 2048
    hop_length = 50  # Hop length for STFT 512
    audio_data, sampling_rate = librosa.load(file_path)
    stft = librosa.stft(audio_data, n_fft=n_fft, hop_length=hop_length)
    # Convert the magnitude spectrogram to decibels (log scale)
    spectrogram = librosa.amplitude_to_db(np.abs(stft))
    # Plot the spectrogram
    plt.figure(figsize=(30, 6))
    librosa.display.specshow(spectrogram, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='linear')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plot = plt.show()
    return plot

def audio_analysis(file_path):
    aw = audio_waveframe(file_path)
    spg = spectrogram(file_path)
    return aw, spg

In [18]:
# meta_data = pl.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv', low_memory=True)
# meta_data.head(5)

In [19]:
#[meta_data.filter((pl.col('primary_label')=='asbfly') & (pl.col('type')=="['song']")).select('filename')]

In [20]:
"""audio_analysis('/kaggle/input/birdclef-2024/train_audio/asbfly/XC374520.ogg')
Audio('/kaggle/input/birdclef-2024/train_audio/asbfly/XC374520.ogg')"""

"audio_analysis('/kaggle/input/birdclef-2024/train_audio/asbfly/XC374520.ogg')\nAudio('/kaggle/input/birdclef-2024/train_audio/asbfly/XC374520.ogg')"

In [22]:
"""with open("/kaggle/input/extracted-features-pickle/extracted_features", "rb") as file:
    pickled_extracted_features = pickle.load(file)"""

'with open("/kaggle/input/extracted-features-pickle/extracted_features", "rb") as file:\n    pickled_extracted_features = pickle.load(file)'

In [23]:
"""# Path to the directory containing your audio dataset
dataset_dir = '/kaggle/input/birdclef-2024/train_audio'
# Initialize an empty dictionary to store the mapping between audio files and labels
label_mapping = {}
# Iterate over subdirectories (classes) in the dataset directory
for label in os.listdir(dataset_dir):
    label_dir = os.path.join(dataset_dir, label)
    # Check if the item in the dataset directory is a directory
    if os.path.isdir(label_dir):
        # Iterate over audio files in the subdirectory (class)
        for audio_file in os.listdir(label_dir):
            # Add the mapping between audio file path and label to the dictionary
            audio_file_path = os.path.join(label_dir, audio_file)
            label_mapping[audio_file_path] = label
            
# label_mapping

# Create a list of tuples containing the audio file paths and labels
data = [(audio_file_path, label) for audio_file_path, label in label_mapping.items()]
# Create a Pandas DataFrame from the list of tuples
annotated_data = pd.DataFrame(data, columns=['audio_file_path', 'label'])

label_encoder = LabelEncoder()
annotated_data['encoded_label'] = label_encoder.fit_transform(annotated_data['label'])

annotated_data.head(10)"""

"# Path to the directory containing your audio dataset\ndataset_dir = '/kaggle/input/birdclef-2024/train_audio'\n# Initialize an empty dictionary to store the mapping between audio files and labels\nlabel_mapping = {}\n# Iterate over subdirectories (classes) in the dataset directory\nfor label in os.listdir(dataset_dir):\n    label_dir = os.path.join(dataset_dir, label)\n    # Check if the item in the dataset directory is a directory\n    if os.path.isdir(label_dir):\n        # Iterate over audio files in the subdirectory (class)\n        for audio_file in os.listdir(label_dir):\n            # Add the mapping between audio file path and label to the dictionary\n            audio_file_path = os.path.join(label_dir, audio_file)\n            label_mapping[audio_file_path] = label\n            \n# label_mapping\n\n# Create a list of tuples containing the audio file paths and labels\ndata = [(audio_file_path, label) for audio_file_path, label in label_mapping.items()]\n# Create a Pandas Dat

In [24]:
"""x = np.vstack(pickled_extracted_features)
y = annotated_data['encoded_label']

print(x.shape)
print(y.shape)"""

"x = np.vstack(pickled_extracted_features)\ny = annotated_data['encoded_label']\n\nprint(x.shape)\nprint(y.shape)"

In [25]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [26]:
"""ros = RandomOverSampler(random_state=42)
features_resampled, labels_resampled = ros.fit_resample(x, y)

print(features_resampled.shape)
print(labels_resampled.shape)"""

'ros = RandomOverSampler(random_state=42)\nfeatures_resampled, labels_resampled = ros.fit_resample(x, y)\n\nprint(features_resampled.shape)\nprint(labels_resampled.shape)'

In [27]:
"""unique_labels = labels_resampled.unique().sort()

x_train, x_test, y_train, y_test = train_test_split(features_resampled, labels_resampled, test_size=0.2, random_state=42)"""

'unique_labels = labels_resampled.unique().sort()\n\nx_train, x_test, y_train, y_test = train_test_split(features_resampled, labels_resampled, test_size=0.2, random_state=42)'

In [28]:
"""from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5, probability=True))
])

poly_kernel_svm_clf.fit(x_train, y_train)"""

'from sklearn.pipeline import Pipeline\nfrom sklearn.svm import SVC\nfrom sklearn.preprocessing import StandardScaler\n\npoly_kernel_svm_clf = Pipeline([\n    ("scaler", StandardScaler()),\n    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5, probability=True))\n])\n\npoly_kernel_svm_clf.fit(x_train, y_train)'

In [29]:
# dump(poly_kernel_svm_clf, 'audio_classifier_svc_deg3poly_model.joblib')

In [30]:
"""from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model = random_forest_classifier.fit(x_train, y_train)
#y_predict = random_forest_model.predict(x_test)"""

'from sklearn.ensemble import RandomForestClassifier\n\nrandom_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)\nrandom_forest_model = random_forest_classifier.fit(x_train, y_train)\n#y_predict = random_forest_model.predict(x_test)'

In [31]:
# y_predict = poly_kernel_svm_clf.predict_proba(x_test)

In [32]:
"""roc_score = roc_auc_score(y_test, y_predict_rf, multi_class='ovo', average='macro', labels=unique_labels)
print("ROC AUC Score:", roc_score)"""

'roc_score = roc_auc_score(y_test, y_predict_rf, multi_class=\'ovo\', average=\'macro\', labels=unique_labels)\nprint("ROC AUC Score:", roc_score)'

In [33]:
"""roc_score = roc_auc_score(y_test, y_predict, multi_class='ovo', average='macro', labels=unique_labels)
print("ROC AUC Score:", roc_score)"""

'roc_score = roc_auc_score(y_test, y_predict, multi_class=\'ovo\', average=\'macro\', labels=unique_labels)\nprint("ROC AUC Score:", roc_score)'

In [34]:
# svc_deg3poly_model = load('/kaggle/input/svc-deg3-poly-clf-model/audio_classifier_svc_deg3poly_model.joblib')

In [35]:
"""from sklearn.calibration import CalibratedClassifierCV

calibrated_clf = CalibratedClassifierCV(svc_deg3poly_model, cv="prefit")
calibrated_clf.fit(x_test, y_test)"""

'from sklearn.calibration import CalibratedClassifierCV\n\ncalibrated_clf = CalibratedClassifierCV(svc_deg3poly_model, cv="prefit")\ncalibrated_clf.fit(x_test, y_test)'

In [36]:
# dump(calibrated_clf, 'calibrated_audio_clf_svc_deg3poly_model.joblib')

In [37]:
calibrated_clf = load('/kaggle/input/calibrated-svc-deg3-poly-clf-model/calibrated_audio_clf_svc_deg3poly_model.joblib')

In [38]:
config = {}
Test = False

if Test:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/'
    config['start'] = 7000
    config['end'] = 8200
    #
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    filenames = [os.path.basename(filename) for filename in filenames_with_path][config['start']:config['end']]
    print(len(filenames))
else:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/test_soundscapes/'
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    filenames = [os.path.basename(filename) for filename in filenames_with_path]
    print(len(filenames))

0


In [39]:
from tqdm import tqdm

chunk_feature_dict = {}

for file in tqdm(filenames):
    full_path = config['root_file_path']+file
    audio, sample_rate = librosa.load(path=full_path, sr=32000)
    cur_file = file.replace('.ogg','')
    samples_per_segment = sample_rate * 5
    if len(audio) != 7680000:
        total_samples = 7680000
    else:
        total_samples = len(audio)
        
    for i in range(0, total_samples+160000, samples_per_segment):
        if i + samples_per_segment <= total_samples:
            segment = audio[i:i + samples_per_segment]
            chunk = cur_file+'_'+str(int((i/32000)+5))
            mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)
            chunk_feature_dict[chunk] = flattened_features

0it [00:00, ?it/s]


In [40]:
meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)
bird_cols = list(meta_data['primary_label'].unique().sort())

submit = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
submit['row_id'] = 'samples'

submit

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
1,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
2,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495


In [41]:
for file_name, feature_set in tqdm(chunk_feature_dict.items()):
    df = pd.DataFrame(calibrated_clf.predict_proba(feature_set), columns=bird_cols)
    # df = pd.DataFrame(svc_deg3poly_model.predict_proba(feature_set), columns=bird_cols)
    df.insert(loc=0, column='row_id', value=file_name)
    submit = pd.concat([submit,df]).reset_index(drop=True)
    
i = submit[(submit.row_id == 'samples')].index
submit = submit.drop(i).reset_index(drop=True)

0it [00:00, ?it/s]


In [42]:
submit.sum(axis=1, numeric_only=True)

Series([], dtype: float64)

In [43]:
sample_submission = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
assert set(sample_submission.columns) == set(submit.columns)

In [44]:
submit.to_csv('submission.csv', index=False)